# 01 Model: Multivariate Analysis of Caregiver Health Outcomes

- Loads the filtered CGUS 2020 data and variable labels.
- Cleans and filters the data to include only completed caregivers (status=1, year=2019).
- Provides descriptive statistics for key variables.
- Generates exploratory plots with readable labels.
- Fits multivariate regression models:
  1. OLS for Physical Strain (Q35)
  2. OLS for Emotional Stress (Q36)
  3. Logistic regression for Isolation (m5c)
- Displays regression tables with coefficients, standard errors, p-values, and confidence intervals

In [ ]:
import sys
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

print(f"Python: {sys.version}")
print(f"pandas: {pd.__version__}")
print(f"statsmodels: {sm.__version__}")
print(f"matplotlib: {plt.matplotlib.__version__}")

In [ ]:
from pathlib import Path
# Determine project root by checking for 'outputs' in cwd or parent
cwd = Path().resolve()
if (cwd / 'outputs').exists():
    root = cwd
elif (cwd.parent / 'outputs').exists():
    root = cwd.parent
else:
    raise FileNotFoundError(f"Cannot find 'outputs' under {cwd} or {cwd.parent}")

print(f"Using project root: {root}")
df = pd.read_csv(root / 'outputs' / 'filtered_caregivers_2020.csv')
labels = pd.read_csv(root / 'outputs' / 'CGUS2020_variable_labels.csv')
labels.columns = labels.columns.str.strip()
label_map = dict(zip(labels['Variable'], labels['Description']))

# Display first rows and shape
df.head(), df.shape

## 2. Data Cleaning & Filtering

In [ ]:
df = df[(df['status'] == 1) & (df['year'] == 2019)].copy()
cols = [
    'Q35', 'Q36', 'm5c',
    'agecgcat', 'sexcg', 'raceCG', 'hispcg',
    'HOURS', 'burden', 'adls', 'iadls',
    'q23g', 'q23i', 'q23j'
]
df = df[cols].dropna()
df['m5c'] = df['m5c'].apply(lambda x: 1 if x in [1, '1', True] else 0)
df.shape

## 3. Descriptive Statistics

In [ ]:
desc_table = df[['HOURS','burden','adls','iadls','Q35','Q36']].describe().T[['mean','std','min','max']]
desc_table

## 4. Exploratory Plots

### 4.1 Physical Strain (Q35) by Burden Level

In [ ]:
x_label = label_map.get('burden', 'burden')
y_label = label_map.get('Q35', 'Physical Strain')
outdir = root / 'outputs' / 'model1'
outdir.mkdir(parents=True, exist_ok=True)
fig, ax = plt.subplots(figsize=(8,5))
df.boxplot(column='Q35', by='burden', ax=ax, patch_artist=True,
           boxprops=dict(facecolor='#8FBBD9'))
plt.suptitle('')
ax.set_title('Physical Strain by Caregiver Burden', fontsize=14)
ax.set_xlabel(x_label + ' (1-5)', fontsize=12)
ax.set_ylabel(y_label + ' (1-5)', fontsize=12)
ax.tick_params(axis='both', labelsize=10)
for i, level in enumerate(sorted(df['burden'].unique()), start=1):
    count = df[df['burden'] == level].shape[0]
    ax.text(i, df['Q35'].max() + 0.3, f'N={count}', ha='center', fontsize=10)
fig.tight_layout()
fig.savefig(outdir / 'box_Q35_by_burden.png', dpi=300)
plt.show()

### 4.2 Isolation Rate (m5c) by Digital Advocacy (q23g)

In [ ]:
adv_map = {1: 'No Advocacy', 2: 'Advocacy', 4: 'Missing/Refused'}
df['advocacy_label'] = df['q23g'].map(adv_map)
fig2, ax2 = plt.subplots(figsize=(8,5))
df.groupby('advocacy_label')['m5c'].mean().plot(kind='bar', color='#D98F8F', ax=ax2)
ax2.set_title('Isolation Rate by Digital Advocacy', fontsize=14)
ax2.set_xlabel('Advocacy Status', fontsize=12)
ax2.set_ylabel('Proportion Feeling Alone', fontsize=12)
ax2.tick_params(axis='x', rotation=45, labelsize=10)
ax2.tick_params(axis='y', labelsize=10)
fig2.tight_layout()
fig2.savefig(outdir / 'bar_iso_by_advocacy.png', dpi=300)
plt.show()

## 5. Regression Modeling

### 5.1 OLS: Physical Strain (Q35)

In [ ]:
mod_phys = smf.ols(
    'Q35 ~ C(agecgcat) + C(sexcg) + C(raceCG) + C(hispcg) + HOURS + burden + adls + iadls + q23g + q23i + q23j',
    data=df
).fit()
mod_phys_params = mod_phys.params.to_frame('coef')
mod_phys_params['se'] = mod_phys.bse
mod_phys_params['pvalue'] = mod_phys.pvalues
mod_phys_params['ci_lower'] = mod_phys.conf_int()[0]
mod_phys_params['ci_upper'] = mod_phys.conf_int()[1]
mod_phys_params = mod_phys_params.round(3)
mod_phys_params

#### 5.1.1 Model Diagnostics for Physical Strain

In [ ]:
resid = mod_phys.resid
fitted = mod_phys.fittedvalues
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(fitted, resid, alpha=0.3)
ax.axhline(0, color='k', linestyle='--')
ax.set_title('Residuals vs Fitted (Physical Strain)')
ax.set_xlabel('Fitted Values')
ax.set_ylabel('Residuals')
fig.tight_layout()
fig.savefig(outdir / 'resid_fitted_phys.png', dpi=300)
plt.show()

import scipy.stats as stats
fig, ax = plt.subplots(figsize=(6,4))
stats.probplot(resid, dist='norm', plot=ax)
ax.set_title('Normal Q-Q Plot (Physical Strain)')
fig.tight_layout()
fig.savefig(outdir / 'qqplot_phys.png', dpi=300)
plt.show()

X = df[['HOURS','burden','adls','iadls','q23g','q23i','q23j']]
X = sm.add_constant(X)
vif_data = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
vif_data

### 5.2 OLS: Emotional Stress (Q36)

In [ ]:
mod_emo = smf.ols(
    'Q36 ~ C(agecgcat) + C(sexcg) + C(raceCG) + C(hispcg) + HOURS + burden + adls + iadls + q23g + q23i + q23j',
    data=df
).fit()
mod_emo_params = mod_emo.params.to_frame('coef')
mod_emo_params['se'] = mod_emo.bse
mod_emo_params['pvalue'] = mod_emo.pvalues
mod_emo_params['ci_lower'] = mod_emo.conf_int()[0]
mod_emo_params['ci_upper'] = mod_emo.conf_int()[1]
mod_emo_params = mod_emo_params.round(3)
mod_emo_params

#### 5.2.1 Model Diagnostics for Emotional Stress

In [ ]:
resid2 = mod_emo.resid
fitted2 = mod_emo.fittedvalues
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(fitted2, resid2, alpha=0.3)
ax.axhline(0, color='k', linestyle='--')
ax.set_title('Residuals vs Fitted (Emotional Stress)')
ax.set_xlabel('Fitted Values')
ax.set_ylabel('Residuals')
fig.tight_layout()
fig.savefig(outdir / 'resid_fitted_emo.png', dpi=300)
plt.show()

fig, ax = plt.subplots(figsize=(6,4))
stats.probplot(resid2, dist='norm', plot=ax)
ax.set_title('Normal Q-Q Plot (Emotional Stress)')
fig.tight_layout()
fig.savefig(outdir / 'qqplot_emo.png', dpi=300)
plt.show()

### 5.3 Logistic Regression: Isolation (m5c)

In [ ]:
mod_iso = smf.logit(
    'm5c ~ C(agecgcat) + C(sexcg) + C(raceCG) + C(hispcg) + HOURS + burden + adls + iadls + q23g + q23i + q23j',
    data=df
).fit()
logit_df = pd.DataFrame({
    'coef': mod_iso.params,
    'se': mod_iso.bse,
    'pvalue': mod_iso.pvalues,
    'ci_lower': mod_iso.conf_int()[0],
    'ci_upper': mod_iso.conf_int()[1],
})
logit_df['OR'] = np.exp(logit_df['coef'])
logit_df['OR_ci_lower'] = np.exp(logit_df['ci_lower'])
logit_df['OR_ci_upper'] = np.exp(logit_df['ci_upper'])
logit_df = logit_df[['coef','se','pvalue','ci_lower','ci_upper','OR','OR_ci_lower','OR_ci_upper']].round(3)
logit_df

#### 5.3.1 Diagnostics: ROC Curve for Isolation Model

In [ ]:
from sklearn.metrics import roc_curve, auc
y_true = df['m5c']
y_pred_prob = mod_iso.predict()
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)
fig, ax = plt.subplots(figsize=(6,5))
ax.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}', color='#4CAF50')
ax.plot([0,1],[0,1],'--', color='gray')
ax.set_title('ROC Curve: Isolation Model', fontsize=14)
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.legend(loc='lower right')
fig.tight_layout()
fig.savefig(outdir / 'roc_isolation.png', dpi=300)
plt.show()

## 6. Results

In [ ]:
mod_phys_params.to_csv(outdir / 'phys_strain_coefs.csv')
mod_emo_params.to_csv(outdir / 'emo_stress_coefs.csv')
logit_df.to_csv(outdir / 'isolation_model_coefs.csv')
desc_table.to_csv(outdir / 'descriptive_stats.csv')
print('Saved all tables and plots to', outdir)